# 第一次作业-预测PM2.5

In [ ]:
#萌新徒手抓瞎

import numpy as np
import pandas as pd

data=pd.read_csv('train.csv')
data=data[data["object-name"]=="PM2.5"]
data[["1","2"]][:5]

## 扒作业答案源码

In [43]:
import sys
import numpy as np
import pandas as pd
import csv

In [44]:
raw_data=np.genfromtxt("train.csv",delimiter=',') #读入文件
data=raw_data[1:,2:] #略去文件头和左侧属性名
where_are_NaNs=np.isnan(data)
data[where_are_NaNs]=0 #将无效值赋为零

month_to_data={} #这个dict有三个维度，月份、污染物和时间

for month in range(12): #数据中包含12个月
    sample=np.empty(shape=(18,480))
    for day in range(20): #每个月内含20天
        for hour in range(24): #污染物有18种，因此每过一天就乘以18
            sample[:,day*24+hour]=data[18*(month*20+day):18*(month*20+day+1),hour]
    month_to_data[month]=sample

In [45]:
x=np.empty(shape=(12*471,18*9),dtype=float)
y=np.empty(shape=(12*471,1),dtype=float)

for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day==19 and hour>14: #471=12*20-10+1
                continue
            
            #reshape(a,b)其中一个参数为-1意味着自适应另一维度来调整，如(1,-1)即变为一行
            #x是取连续9小时内18种污染物的数值作为原始数据，且化为一行进行储存。两个维度分别是时间和污染物。
            #y是取对应的第十小时PM2.5的数值作为结果
            x[month*471+day*24+hour,:]=month_to_data[month][:,day*24+hour:day*24+hour+9].reshape(1,-1)
            y[month*471+day*24+hour,0]=month_to_data[month][9,day*24+hour+9] #PM2.5在第九行

            #总结一下上面这样处理的合理之处：所有连续9小时都被囊括，包括了跨天的时间段，最大化利用数据
            #一切都归结于reshape的机智操作，赞！
            
# pd.DataFrame(x)
#以前两行为例进行解释
#第0行-第一种污染物从0点开始连续九小时的数值-第二种污染物从0点开始连续九小时的数值-
#第1行-第一种污染物从1点开始连续九小时的数值-第二种污染物从1点开始连续九小时的数值-
#以此类推就得到了训练用的dataset

In [4]:
#这种归一化方法不优，在kaggle上结果较差

mean=np.mean(x,axis=0) #dataset中每一列都是同种污染物的数值
std=np.std(x,axis=0)

for i in range(x.shape[0]): #x.shape[0]=12*471
    for j in range(x.shape[1]): #x.shape[1]=18*9
        x[i][j]=(x[i][j]-mean[j])/(std[j]+0.0001) #归一化处理

In [46]:
#这种方法在kaggle上结果更好

Max=np.max(x,axis=0)
Min=np.min(x,axis=0)

for i in range(x.shape[1]):
    x[:,i]=(x[:,i]-Min[i])/(Max[i]-Min[i])

In [47]:
dim=x.shape[1]+1
w=np.zeros(shape=(dim,1))
W=np.zeros(shape=(dim,1))
x=np.concatenate((np.ones((x.shape[0],1)),x),axis=1).astype(float) #左侧拼接bias项所对应的常数1
learning_rate=np.array([[200]]*dim)
adagrad_w=np.zeros(shape=(dim,1))
adagrad_W=np.zeros(shape=(dim,1))

#下面就是进行Gradient Descent
#矢量化操作需要常记的一些点
#1.dataset中的实例作为行，即一行一个实例，然后参数作为列，或者调转，最好形成固定习惯
#2.记住计算平方就是A.T@A，即转置后直接矩阵乘法
#3.如果直接在dataset左侧拼接常数1，那么各种操作都会方便许多

x_train=x
y_train=y

for T in range(5000):
    if T%50==0:
        print("T=",T)
        print("Loss:%f" % (np.sqrt(np.sum(np.power(y_train-x_train@w-np.power(x_train,2)@W,2))/x_train.shape[0])))

    gradient_w=(-2)*np.transpose(x_train)@(y_train-x_train@w-np.power(x_train,2)@W)
    gradient_W=(-2)*np.transpose(np.power(x_train,2))@(y_train-x_train@w-np.power(x_train,2)@W)
    adagrad_w+=gradient_w**2
    adagrad_W+=gradient_W**2
    
    w-=learning_rate/np.sqrt(adagrad_w+0.00001)*gradient_w
    W-=learning_rate/np.sqrt(adagrad_W+0.00001)*gradient_W

np.save('weight1.npy',w) #训练得到的权重存储在特定后缀文件.npy
np.save("weight2.npy",W)

T= 0
Loss:27.071215
T= 50
Loss:10.145009
T= 100
Loss:9.390506
T= 150
Loss:8.988748
T= 200
Loss:8.707165
T= 250
Loss:8.489133
T= 300
Loss:8.311466
T= 350
Loss:8.162098
T= 400
Loss:8.033809
T= 450
Loss:7.921869
T= 500
Loss:7.822978
T= 550
Loss:7.734726
T= 600
Loss:7.655295
T= 650
Loss:7.583275
T= 700
Loss:7.517555
T= 750
Loss:7.457242
T= 800
Loss:7.401611
T= 850
Loss:7.350068
T= 900
Loss:7.302119
T= 950
Loss:7.257352
T= 1000
Loss:7.215417
T= 1050
Loss:7.176019
T= 1100
Loss:7.138906
T= 1150
Loss:7.103859
T= 1200
Loss:7.070689
T= 1250
Loss:7.039231
T= 1300
Loss:7.009340
T= 1350
Loss:6.980889
T= 1400
Loss:6.953765
T= 1450
Loss:6.927866
T= 1500
Loss:6.903103
T= 1550
Loss:6.879396
T= 1600
Loss:6.856670
T= 1650
Loss:6.834860
T= 1700
Loss:6.813907
T= 1750
Loss:6.793755
T= 1800
Loss:6.774355
T= 1850
Loss:6.755662
T= 1900
Loss:6.737634
T= 1950
Loss:6.720232
T= 2000
Loss:6.703421
T= 2050
Loss:6.687169
T= 2100
Loss:6.671445
T= 2150
Loss:6.656221
T= 2200
Loss:6.641472
T= 2250
Loss:6.627173
T= 2300
L

KeyboardInterrupt: 

In [38]:
w=np.load('weight1.npy')
W=np.load('weight2.npy')
test_raw_data=np.genfromtxt('test.csv',delimiter=',')
test_data=test_raw_data[1:,2:]
where_are_NaNs=np.isnan(test_data)
test_data[where_are_NaNs]=0

In [41]:
test_x=np.empty(shape=(240,18*9),dtype=float)

for i in range(240):
    test_x[i,:]=test_data[i*18:(i+1)*18,:].reshape(1,-1)
    
# mean=np.mean(test_x,axis=0)
# std=np.std(test_x,axis=0)
Max=np.max(test_x,axis=0)
Min=np.min(test_x,axis=0)

for i in range(test_x.shape[1]):
    test_x[:,i]=(test_x[:,i]-Min[i])/(Max[i]-Min[i])
            
test_x=np.concatenate((np.ones(shape=(test_x.shape[0],1)),test_x),axis=1).astype(float)
answer=test_x.dot(w)+np.power(test_x,2)@W
answer

array([[  8.88556148],
       [ 17.1127874 ],
       [ 11.96803945],
       [ 11.33690944],
       [ 25.46145054],
       [ 18.79196751],
       [ 19.4308737 ],
       [ 28.20506057],
       [ 18.98652906],
       [ 52.59117189],
       [ 15.63123066],
       [ 11.89337473],
       [ 50.11259948],
       [ 42.80364196],
       [ 16.25296616],
       [ 10.26948773],
       [ 28.35981395],
       [ 56.44011898],
       [  0.52654042],
       [ 12.26332544],
       [ 36.9802351 ],
       [ 66.7846933 ],
       [  8.28073646],
       [ 19.18854086],
       [ 12.51190422],
       [ 32.45111504],
       [ 12.38585988],
       [ 52.67068377],
       [  7.64995832],
       [ 45.69410671],
       [ 21.45766155],
       [ 10.39036783],
       [  6.84029922],
       [ 21.82025887],
       [ 24.26241823],
       [ 30.07537388],
       [ 37.02479951],
       [ 24.46883364],
       [ 38.93819436],
       [ 32.72573687],
       [  4.26985846],
       [ 30.37939524],
       [ 30.27118213],
       [ 41

In [42]:
f=open("ans.csv","w")
ws=csv.writer(f)
title=['id','value']
ws.writerow(title)

for i in range(240):
    content=['id_'+str(i),answer[i][0]]
    ws.writerow(content)

## z君独立尝试

In [ ]:
import numpy as np
import sys
import csv

data_raw=np.genfromtxt("train.csv",delimiter=',')
data=data_raw[1:,2:]
where_are_NaNs=np.isnan(data)
data[where_are_NaNs]=0

In [ ]:
con_hour_data=[]

for i in range(240):
    con_hour_data.append(data[i*18+9,:])

con_hour_data=np.array(con_hour_data).reshape(1,-1)

x_train=np.zeros((5750,9)) #这里需要非常注意，如果有列向量之类的，不要使用np.array方法进行转换，而是直接用np.zeros方法创建！
y_train=np.zeros((5750,1))

for i in range(con_hour_data.shape[1]-10):
    x_train[i,0:9]=con_hour_data[0,i:i+9]
    y_train[i]=con_hour_data[0,i+9]

In [ ]:
mean=np.mean(x_train,axis=1)
std=np.std(x_train,axis=1)

# print(mean.shape)

for i in range(x_train.shape[0]):
    x_train[i]=(x_train[i]-mean[i])/(std[i]+0.0001)
    
w=np.zeros((10,1))
W=np.zeros((10,1))
x_train=np.concatenate((np.ones((x_train.shape[0],1)),x_train),axis=1).astype(float)

In [ ]:
learning_rate=np.array([[100]]*10)
adagrad_w=np.zeros((10,1))
adagrad_W=np.zeros((10,1))

for T in range(10000):
    if T%500==0:
        print("T:%d" % T)
        print("Loss:%f" % (np.sqrt(np.sum(np.power(y_train-x_train@w-np.power(x_train,2)@W,2))/x_train.shape[0])))
    
    gradient_w=(-2)*np.transpose(x_train)@(y_train-x_train@w-np.power(x_train,2)@W)
    gradient_W=(-2)*np.transpose(np.power(x_train,2))@(y_train-x_train@w-np.power(x_train,2)@W)
    adagrad_w+=gradient_w**2
    adagrad_W+=gradient_W**2
    
    w-=learning_rate/np.sqrt(adagrad_w+0.00001)*gradient_w
    W-=learning_rate/np.sqrt(adagrad_W+0.00001)*gradient_W
    
np.save("weight1.npy",w)
np.save("weight2.npy",W)